In [1]:
from kaggle_environments import make
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from utils import Features, ReplayBuffer, NetModel

No pygame installed, ignoring import


In [223]:
# agent
class SelectActionNet:
    def __init__(self, feature_estimate):
        self.feature = feature_estimate

    def select_action(self):
        features = self.feature

        c_interval_length = features.expected.mean() * 0.35 + 0.05
        interval_score = c_interval_length * features.interval_length

        # c_trace = 0.15 * features.expected
        # trace_score = features.opp_acc_trace * c_trace

        rand = np.random.random(features.n_bandits) * 0.0001

        step_feature = np.full(features.n_bandits, features.step/2000)
        data_feature = np.stack((step_feature, feature.expected, feature.pro_lower_bound, feature.pro_upper_bound, feature.interval_length, feature.num_selection/25, feature.num_opp_selection/25, feature.sum_num/50, feature.num_ratio, feature.ratio_reward, feature.get_reward_th, feature.num_opp_before/10, feature.num_before/10, feature.num_opp_after/10, feature.num_after/10, feature.opp_bunching, feature.opp_rep_trace), axis=1)
        
        # print(data_feature.shape)
        net_expect_value = network_model.get_value(data_feature)

        # scores = net_expect_value + interval_score + trace_score + rand
        scores = net_expect_value + interval_score + rand

        # select_action = int(np.argmax(scores))
        
        # print(net_expect_value.shape)
        
        # select_action = int(np.argmax(net_expect_value))

        # return select_action
        
        return scores

In [215]:
def agent(observation, configuration):
    global feature, selector, total_reward

    player = observation.agentIndex
    opponent = 1 - player
    step = observation.step
    new_reward = observation.reward

    if step == 0:
        feature = Features(configuration.banditCount)
        selector = SelectActionNet(feature)
        total_reward = 0
    else:
        action = observation.lastActions[player]
        opp_action = observation.lastActions[opponent]
        reward = new_reward - total_reward

        feature_value = (feature.step/2000, feature.expected[action], feature.pro_lower_bound[action], feature.pro_upper_bound[action],feature.interval_length[action], feature.num_selection[action]/25, feature.num_opp_selection[action]/25, feature.sum_num[action]/50, feature.num_ratio[action], feature.ratio_reward[action], feature.get_reward_th[action], feature.num_opp_before[action]/10, feature.num_before[action]/10, feature.num_opp_after[action]/10, feature.num_after[action]/10, feature.opp_bunching[action], feature.opp_rep_trace[action], reward)
        replay_buffer.push(feature_value)

        feature.feature_update(step, action, reward, opp_action)
        total_reward = new_reward

    action = selector.select_action()
    return action

In [275]:
def agent(observation, configuration):
    global feature, selector, total_reward, policy_params

    player = observation.agentIndex
    opponent = 1 - player
    step = observation.step
    new_reward = observation.reward

    if step == 0:
        feature = Features(configuration.banditCount)
        selector = SelectActionNet(feature)
        total_reward = 0
        policy_params = np.zeros(100)
    else:
        action = observation.lastActions[player]
        opp_action = observation.lastActions[opponent]
        reward = new_reward - total_reward

        feature_value = (feature.step/2000, feature.expected[action], feature.pro_lower_bound[action], feature.pro_upper_bound[action],feature.interval_length[action], feature.num_selection[action]/25, feature.num_opp_selection[action]/25, feature.sum_num[action]/50, feature.num_ratio[action], feature.ratio_reward[action], feature.get_reward_th[action], feature.num_opp_before[action]/10, feature.num_before[action]/10, feature.num_opp_after[action]/10, feature.num_after[action]/10, feature.opp_bunching[action], feature.opp_rep_trace[action], reward)
        replay_buffer.push(feature_value)

        feature.feature_update(step, action, reward, opp_action)
        total_reward = new_reward

        score = selector.select_action().reshape(100,)

        policy_params = 0.1 * (score - np.mean(score)) / np.std(score) + 0.9 * policy_params

    action_probs = np.exp(policy_params) / np.sum(np.exp(policy_params))

    # action = np.random.choice(100, p=action_probs)
    if observation.step > 100:
        action = int(np.argmax(action_probs))
    else:
        action = np.random.choice(100, p=action_probs)
    
    # action = selector.select_action()
    return action

In [4]:
def agent_opp(observation, configuration):
    global feature_opp, selector_opp, total_reward_opp

    player = observation.agentIndex
    opponent = 1 - player
    step = observation.step
    new_reward = observation.reward

    if step == 0:
        feature_opp = Features(configuration.banditCount)
        selector_opp = SelectActionNet(feature_opp)
        total_reward_opp = 0
    else:
        action = observation.lastActions[player]
        opp_action = observation.lastActions[opponent]
        reward = new_reward - total_reward_opp

        feature_value = (feature_opp.step/2000, feature_opp.expected[action], feature_opp.pro_lower_bound[action], feature_opp.pro_upper_bound[action],feature_opp.interval_length[action], feature_opp.num_selection[action]/25, feature_opp.num_opp_selection[action]/25, feature_opp.sum_num[action]/50, feature_opp.num_ratio[action], feature_opp.ratio_reward[action], feature_opp.get_reward_th[action], feature_opp.num_opp_before[action]/10, feature_opp.num_before[action]/10, feature_opp.num_opp_after[action]/10, feature_opp.num_after[action]/10, feature_opp.opp_bunching[action], feature_opp.opp_rep_trace[action], reward)

        replay_buffer.push(feature_value)

        feature_opp.feature_update(step, action, reward, opp_action)
        total_reward_opp = new_reward

    action = selector_opp.select_action()
    return action

In [217]:
class ClasssfyNN(nn.Module):
    """ 定义神经网络 """

    def __init__(self, input_size, hidden_size1=16, hidden_size2=16):
        super(ClasssfyNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        # out = self.relu1(out)
        out = nn.functional.tanh(out)
        out = self.fc2(out)
        # out = self.relu2(out)
        out = nn.functional.tanh(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
    
        # out = nn.functional.softmax(out, dim=-1)
        return out


class ReplayBuffer:
    """ 定义经验回放缓冲区 """

    def __init__(self, capacity=4000):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, transition):
        if len(self.buffer) < self.capacity:
            self.buffer.append(transition)
        else:
            self.buffer[self.position] = transition
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def refresh(self):
        self.buffer = []
        self.position = 0


class NetModel:
    def __init__(self, input_size):
        self.input_size = input_size
        self.network = ClasssfyNN(input_size)

    def get_value(self, x):
        """ 输入x应为矩阵形式 """
        inputs = torch.from_numpy(x.astype(np.float32))
        outputs = self.network(inputs)
        y = outputs.detach().numpy().T
        return y

    def train(self, data, lr=0.01, num_epochs=100):
        """ 输入训练数据为ReplayBuffer中的buffer列表 """
        # 处理训练数据
        data_train = np.array(data)
        x_train = torch.from_numpy(data_train[:, 0:self.input_size].astype(np.float32))
        y_train = torch.from_numpy(data_train[:, self.input_size:(self.input_size + 1)].astype(np.float32))

        # 定义损失函数和优化器
        criterion = nn.BCELoss()
        # criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.network.parameters(), lr=lr)
        # optimizer = optim.SGD(self.network.parameters(), lr=lr)

        # 训练模型
        for epoch in range(num_epochs):
            # 前向传播
            # print(x_train.shape)
            outputs = self.network(x_train)#.reshape(100,)
            # print(outputs.shape)
            loss = criterion(outputs, y_train)
            # print(loss.item())
            # 反向传播与优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [241]:
# 经验回放缓冲区以收集样本
replay_buffer = ReplayBuffer()
# 神经网络模型
# network_model = NetModel(17)
# 训练
for i in range(40):
    print("epoch", i)
    env = make("mab", debug=True)
    if i % 2 == 0:
        env.run([agent, "bay_sub.py"])
        # env.run([agent, "neil_and_chris.py"])
    elif i % 2 == 1:
        env.run([agent, "Egreedysubmission.py"])
    # if i % 3 == 0:
    #     env.run([agent, "bay_sub.py"])
    # elif i % 3 == 1:
    #     env.run([agent, "Egreedysubmission.py"])
    # elif i % 3 == 2:
    #     env.run([agent, "ucb.py"])
    network_model.train(replay_buffer.buffer)
    # network_model.train(replay_buffer.sample(500))
    replay_buffer.refresh()

epoch 0


KeyboardInterrupt: 

In [236]:
# 测试
env = make("mab", debug=True)
env.run(["bay_sub.py", "main.py"])
env.render(mode="ipython", width=800, height=800)

TypeError: '<' not supported between instances of 'str' and 'int'

In [289]:
# 评估
def print_rounds(file1, file2, N=10):
    env = make("mab", debug=True)

    for i in range(N):
        env.run([file1, file2])
        p1_score = env.steps[-1][0]['reward']
        p2_score = env.steps[-1][1]['reward']
        env.reset()
        print(f"Round {i+1}: {p1_score} - {p2_score}")
# "bay_sub.py"
# print_rounds("test_softmax_9.py", agent)
print_rounds("bay_sub.py", "test_softmax_9.py")
# print_rounds("ucb.py", agent)
# print_rounds("neil_and_chris.py", agent)
# print_rounds("Egreedysubmission.py", agent)

Round 1: 628 - 651
Round 2: 601 - 576
Round 3: 664 - 606
Round 4: 652 - 595
Round 5: 654 - 600
Round 6: 663 - 692
Round 7: 605 - 592
Round 8: 651 - 652
Round 9: 684 - 682
Round 10: 591 - 613


In [91]:
torch.save(network_model.network.state_dict(), "test_1201_4.pth")

In [242]:
network_model = NetModel(17)
state_dict = torch.load("test_1201_2.pth")
network_model.network.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
def print_rounds(file1, file2, N=5):
    env = make("mab", debug=True)

    for i in range(N):
        env.run([file1, file2])
        p1_score = env.steps[-1][0]['reward']
        p2_score = env.steps[-1][1]['reward']
        env.reset()
        print(f"Round {i+1}: {p1_score} - {p2_score}")

print_rounds("ucb.py", agent)

In [ ]:
from scipy.stats import beta

def bay_agent(observation, configuration):
    global total_reward, bandit, post_a, post_b, feature, selector

    n_bandits = configuration.banditCount
    player = observation.agentIndex
    opponent = 1 - player
    step = observation.step
    new_reward = observation.reward

    if observation.step == 0:
        feature = Features(configuration.banditCount)
        selector = SelectActionNet(feature)
        total_reward = 0
        post_a = np.ones(n_bandits)
        post_b = np.ones(n_bandits)

    else:
        r = observation.reward - total_reward
        # total_reward = observation.reward

        post_a[bandit] += r + (1 - observation.step / 2000)
        post_b[bandit] += (1 - r)

        action = observation.lastActions[player]
        opp_action = observation.lastActions[opponent]
        reward = new_reward - total_reward


        feature_value = (feature.step/2000, feature.expected[action], feature.pro_lower_bound[action], feature.pro_upper_bound[action],feature.interval_length[action], feature.num_selection[action]/25, feature.num_opp_selection[action]/25, feature.sum_num[action]/50, feature.num_ratio[action], feature.ratio_reward[action], feature.get_reward_th[action], feature.num_opp_before[action]/10, feature.num_before[action]/10, feature.num_opp_after[action]/10, feature.num_after[action]/10, feature.opp_bunching[action], feature.opp_rep_trace[action], reward)
        replay_buffer.push(feature_value)

        feature.feature_update(step, action, reward, opp_action)
        total_reward = new_reward

    bound = post_a / (post_a + post_b).astype(float) + beta.std(post_a, post_b) * 3
    bandit = int(np.argmax(bound))

    return bandit


In [ ]:
# 经验回放缓冲区以收集样本
replay_buffer = ReplayBuffer()
# 神经网络模型
network_model = NetModel(17)
# 训练
for i in range(100):
    print("epoch", i)
    env = make("mab", debug=True)
    env.run([bay_agent, "bay_sub.py"])
    network_model.train(replay_buffer.buffer)
    replay_buffer.refresh()

In [ ]:
# 测试
env = make("mab", debug=True)
env.run([bay_agent, "ucb.py"])
env.render(mode="ipython", width=800, height=800)